In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('..')

In [69]:
from src.datamodules.celeba import CelebaDataset
import matplotlib.pyplot as plt
import json
from src.models.celeba.text_modules import CelebaTextEncoder, CelebaTextDecoder, CelebaTextClassifier
from src.models.celeba.img_modules import CelebaImgClassifier
import torch
from torch.distributions import OneHotCategorical

In [70]:
net = CelebaTextClassifier()

In [71]:
output = net(torch.randn(5, 256, 71))

In [72]:
output.shape

torch.Size([5, 40])

In [66]:
net = CelebaImgClassifier()

In [67]:
output = net(torch.randn(5, 3, 64, 64))

In [68]:
output.shape

torch.Size([5, 40])

In [41]:
dataset = CelebaDataset("data", partition=1)

[autoreload of pytorch_lightning.utilities.distributed failed: Traceback (most recent call last):
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/site-packages/IPython/e

In [42]:
img, text_str, label = dataset[0]

In [43]:
text_str.shape

torch.Size([256, 71])

In [45]:
decoder = CelebaTextDecoder()

In [46]:
output = decoder(torch.randn(5, 32))

In [48]:
llh = OneHotCategorical(output)

In [61]:
llh.sample().shape

torch.Size([5, 256, 71])

In [62]:
256 * 71

18176

In [63]:
3 * 64 * 64

12288

In [50]:
llh.log_prob(text_str).shape

torch.Size([5, 256])

In [58]:
llh.mean.shape

torch.Size([5, 256, 71])

In [59]:
llh.mean.sum(-1)

tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
       grad_fn=<SumBackward1>)

In [ ]:
OneHotCategorical(logits).sample(torch.Size([num_samples]))

In [15]:
with open("data/CelebA/alphabet.json") as alphabet_file:
    alphabet = str("".join(json.load(alphabet_file)))

[autoreload of src.models.celeba failed: Traceback (most recent call last):
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/yizhe/miniconda3/envs/vi-lab/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/yizhe/code/vi-lab/src/models/celeba/__init__.py", line 1, in <module>
    from .img_modules import CelebaImgDec

In [12]:
len(dataset)

19867

In [2]:
import torch
import torch.nn as nn
from src.utils import load_yaml
from src.experiments import (
    HierPMVAE_v1_Experiment,
    Fusion_MVAE_Experiment
)
from src.models.encoders_decoders.base import (
    DeepSVHNEncoder
)

In [17]:
config_path = "configs/tmp/vaevae_6.yaml"
hparams = load_yaml(config_path)

In [18]:
expt = Fusion_MVAE_Experiment(hparams)

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [19]:
sum(p.numel() for p in expt.model.parameters() if p.requires_grad)

2135835

In [16]:
sum(p.numel() for p in expt.model.parameters() if p.requires_grad)

2122875

In [3]:
config_path = "configs/mnist_svhn/hier_pmvae_v1.yaml"
hparams = load_yaml(config_path)

In [4]:
expt = HierPMVAE_v1_Experiment(hparams)

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [116]:
B = 5
K = 3
m_latent_dim = 4
s_latent_dim = 16

inputs = [
    torch.randn(B, 1, 28, 28),
    # None,
    torch.randn(B, 3, 32, 32)
]

In [19]:
from src.models.vdvae.encoder_decoder import (
    Encoder,
    Decoder,
    parse_layer_string
)
import torch
import numpy as np

In [13]:
x = torch.randn(5, 32, 32, 3)
x_target = torch.randn(5, 32, 32, 3)

In [5]:
encoder = Encoder()
decoder = Decoder()

In [8]:
activations = encoder.forward(x)

In [9]:
px_z, stats = decoder.forward(activations)

In [14]:
distortion_per_pixel = decoder.out_net.nll(px_z, x_target)

In [20]:
rate_per_pixel = torch.zeros_like(distortion_per_pixel)
ndims = np.prod(x.shape[1:])
for statdict in stats:
    rate_per_pixel += statdict['kl'].sum(dim=(1, 2, 3))
rate_per_pixel /= ndims

In [23]:
px_z = decoder.forward_uncond(5, t=0.8)

In [25]:
out = decoder.out_net.sample(px_z)

# Test Methods

In [51]:
latents = model.encode(inputs)

In [52]:
latents['s'].shape

torch.Size([5, 16])

In [53]:
[o.shape for o in latents['m']]

AttributeError: 'NoneType' object has no attribute 'shape'

In [54]:
mnist, svhn = model.decode(latents, mean=True)

In [55]:
mnist.shape, svhn.shape

(torch.Size([5, 1, 28, 28]), torch.Size([5, 3, 32, 32]))

In [42]:
mnist, svhn = model.sample(K, mean=True)

In [43]:
mnist.shape, svhn.shape

(torch.Size([3, 1, 28, 28]), torch.Size([3, 3, 32, 32]))

In [57]:
mnist, svhn = model.cross_reconstruct(inputs)

In [58]:
mnist.shape, svhn.shape

(torch.Size([5, 1, 28, 28]), torch.Size([5, 3, 32, 32]))

# Test Log Prob

In [113]:
log_prob, latent, context = model.log_q_z_x(inputs, num_samples=1)

In [88]:
latent['s'].shape

torch.Size([15, 16])

In [89]:
[l.shape for l in latent['m']]

[torch.Size([15, 4]), torch.Size([15, 4])]

In [91]:
context['s'].shape

torch.Size([5, 32])

In [90]:
[c.shape for c in context['m']]

[torch.Size([15, 24]), torch.Size([15, 24])]

In [92]:
log_prob = model.log_q_z_x(latent=latent, context=context)

ValueError: Number of input items must be equal to number of context items.

In [93]:
log_prob.shape

torch.Size([15])

In [95]:
log_prob = model.log_p_z(latent)

In [96]:
log_prob.shape

torch.Size([5])

In [114]:
log_prob = model.log_p_x_z(inputs, latent, [1.0, 1.0], num_samples=1)

torch.Size([5, 1, 28, 28])
torch.Size([5, 20])


In [115]:
log_prob.shape

torch.Size([5])

# Test Objectives

In [117]:
elbo = expt.obj(
    model,
    inputs,
    [1.0, 1.0],
    kl_multiplier=1.0
)

In [118]:
elbo.shape

torch.Size([5])

In [121]:
output = expt.test_step({"data": inputs}, 0)

In [122]:
output.shape

torch.Size([])